In [66]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
import warnings
import sys
import os
warnings.filterwarnings('ignore')

In [59]:
sys.path.append(os.path.abspath("..\src"))
from utils.ClaseAbdon import DataFrameAnalyzer

In [60]:
tienda = pd.read_csv(r'..\data\raw\Merch_RAW.csv')

In [61]:
df = tienda.copy()

In [62]:
df['Order_Date'] = pd.to_datetime(df['Order_Date'])

In [63]:
df.set_index('Order_ID',inplace=True)

In [64]:
df_ob = df.select_dtypes('object')
for col in df_ob.columns:
    print(f'==========={col}===========')
    print(f'Numero de unicos: {df[col].nunique()}')
    print(f'Valores Unicos: {df[col].unique()}')
    print('\n')

===========Product_ID===========
Numero de unicos: 13
Valores Unicos: ['BF1543' 'BF1544' 'BF1545' 'BF1546' 'BF1547' 'BF1548' 'BF1549' 'BF1550'
 'BF1551' 'BF1552' 'BF1553' 'BF1554' 'BF1555']


===========Product_Category===========
Numero de unicos: 3
Valores Unicos: ['Clothing' 'Other' 'Ornaments']


===========Buyer_Gender===========
Numero de unicos: 2
Valores Unicos: ['Male' 'Female']


===========Order_Location===========
Numero de unicos: 25
Valores Unicos: ['New Jersey' 'Las Vegas' 'Cardiff' 'Pittsburgh' 'Miami' 'Sydney'
 'Memphis' 'New York' 'Montreal' 'Sacramento' 'Paris' 'San Antonio'
 'Cleveland' 'London' 'Portland' 'Detroit' 'Dublin' 'Glasgow' 'Austin'
 'Toronto' 'Mumbai' 'San Francisco' 'Manchester' 'Liverpool' 'New Delhi']


===========International_Shipping===========
Numero de unicos: 2
Valores Unicos: ['No' 'Yes']




In [65]:
label = df[['Product_ID','Order_Location','International_Shipping','Buyer_Gender']]
label_encoder = LabelEncoder()

for col in label.columns:
    label[col] = label_encoder.fit_transform(label[col])
df.drop(columns=['Product_ID','Order_Location','International_Shipping','Buyer_Gender'],inplace=True)

df = pd.concat([df,label],axis=1)

In [68]:
df['Product_Category']

Order_ID
189440     Clothing
187385     Clothing
181844        Other
197934        Other
122470        Other
            ...    
136943     Clothing
172276     Clothing
160191    Ornaments
195268        Other
138160        Other
Name: Product_Category, Length: 7394, dtype: object

In [69]:
one = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
df[['Product_Category']] = one.fit_transform(df[['Product_Category']])
df.drop(columns=['Product_Category'])

ValueError: Columns must be same length as key